# Sulphur Isotope Model

## General definitions

In [1]:
import numpy as np
import pandas as pd
import ipywidgets as wdg
from plotly_default import go, graph_config, sel_trace
from plotly.subplots import make_subplots
from scipy.integrate import solve_bvp
import random
# import plotly.io as pio

In [2]:
# Conversion between isotope ratios and delta notation

VCDT = 0.0441626 # vienna canyon diablo troilite standard 34S/32S ratio

def delta_to_R(delta, std=VCDT):
    '''Takes in a d34S; outputs a 34S/32S ratio'''
    return std * ((delta/1000) + 1)

def R_to_delta(R, std=VCDT):
    '''Takes in a 34S/32S ratio; outputs the d34S'''
    return 1000*((R/std) - 1)

def wt_to_M(wt, Mr=55.845, density=1.7):
    '''Converts a weight percent of a substance with molar mass Mr (default iron) into a molar concentration
    in a sediment of specified density (default 1.7 g/cm³)'''
    # g/dm³  = 1000 * g/cm³ = 1000 * density * wt% / 100 = 10 * density * wt%
    return 10. * density * wt / Mr

def M_to_wt(M, Mr=55.845, density=1.7):
    return M * Mr / (10. * density)

def ratio_to_concs(R, C):
    '''Takes a ratio of two concentrations R = a/b
    and a total concentration C = a + b
    and returns an (a, b) tuple'''
    return (C/(1 + 1/R) , C/(1+R))

# The number of seconds in 1 thousand years
s_in_ka = 1000 * 365.24 * 24 * 60 * 60

In [3]:
# Read in real data
log_df=pd.read_csv('./Data/Log_data_combined.csv', index_col=0)

cat_symbols = {'Mudstone': 'circle',
               'Carbonate': 'star',
               'Pervasively pyritised regions': 'square',
               'Bed 22': 'diamond'}
categories = list(cat_symbols.keys())

# log_df

In [4]:
# Set up widgets
w_df = pd.read_csv('./Data/GDE_parameters_v2.csv')


def create_sliders(w_df, prefix='w'):
    w_dict = {}

    for w in w_df.iterrows():

        # Make a new variable for each widget
        vars()[prefix+'_'+w[1].var_name] = wdg.FloatSlider(value = w[1].value,
                                                     min = w[1].start,
                                                     max = w[1].stop,
                                                     step = w[1].step,
                                                     description = w[1].display_name,
                                                     layout=wdg.Layout(width='100%')
                                                    )

        # Add to a dictionary of original variables and widgets
        w_dict[w[1].var_name] = vars()[prefix+'_'+w[1].var_name]
    
    return w_dict

## Solving the steady-state General Diagenetic Equation

$y$  | represents   | <div style="width:100px">value at $z=0$</div> | <div style="width:100px"> value at $z=z_\mathrm{max}$</div>
-----|--------------------------------------------------|--------------------------|-----------------------------
$y_0$|$[\mathrm{^{34}SO_4}]$                            |$[\mathrm{^{34}SO_4}]_0$  |?? $(\approx 0)$
$y_1$|$\frac{\partial[\mathrm{^{34}SO_4}]}{\partial z}$ |??                        |$0$
$y_2$|$[\mathrm{^{32}SO_4}]$                            |$[\mathrm{^{32}SO_4}]_0$  |?? $(\approx 0)$
$y_3$|$\frac{\partial[\mathrm{^{32}SO_4}]}{\partial z}$ |??                        |$0$
$y_4$|$[\mathrm{Fe^{34}S_2}]$                           |$[\mathrm{Fe^{34}S_2}]_0$ |??
$y_5$|$[\mathrm{Fe^{32}S_2}]$                           |$[\mathrm{Fe^{32}S_2}]_0$ |??

$$\frac{\partial y_0}{\partial z} = y_1$$

$$\frac{\partial y_1}{\partial z} = \left( vy_1 + \frac{k_\mathrm{MSR}\left(y_0 y_2 + y_0^2\right)}{\frac{y_2}{\alpha} + y_0} \right)/D_{34}$$

$$\frac{\partial y_2}{\partial z} = y_3$$

$$\frac{\partial y_3}{\partial z} = \left( vy_3 + \frac{k_\mathrm{MSR}\left(y_2^2 + y_0 y_2\right)}{\alpha y_0 + y_2} \right)/D_{32}$$

$$\frac{\partial y_4}{\partial z} = \frac{k_\mathrm{MSR}\left(y_0 y_2 + y_0^2\right)}{2v\left(\frac{y_2}{\alpha} + y_0\right)}$$

$$\frac{\partial y_5}{\partial z} = \frac{k_\mathrm{MSR}\left(y_2^2 + y_0 y_2\right)}{2v\left(\alpha y_0 + y_2\right)}$$

In [5]:
def solve_GDE(zs, SO4_0, py_0, d34S_SO4_0, epsilon_MSR, k_MSR, beta, D_32, v):
    '''Solves the system of general diagenetic equations listed above,
    for the array of depths in zs, and returns a 2D array of y values
    (rows for each yi; cols are values of yi for each z)'''
    
    R_SO4_0 = delta_to_R(d34S_SO4_0) 
    a_MSR = delta_to_R(d34S_SO4_0 - epsilon_MSR)/R_SO4_0 # MSR fractionation factor    
    R_py_0 = delta_to_R(d34S_SO4_0)*a_MSR # add ability to tweak in future?
    
    # weight-dependent diffusion coefficients (Wortmann & Chernyavsky, 2011)
    D_34 = D_32 * (98/96)**(-beta)
    
    SO4_34_0 , SO4_32_0 = ratio_to_concs(R_SO4_0, SO4_0)
    py_34_0  , py_32_0  = ratio_to_concs(R_py_0, py_0)
    
    # Define the ODEs and boundary conditions for the chosen parameters
    def SS_ODEs(x,y):
        '''The right-hand side of the steady state general diagenetic equations'''
        return np.vstack([
            y[1,:], # = dy[0,:]/dz
            (v*y[1,:] + k_MSR*(y[0,:]*y[2,:] + y[0,:]**2)/((y[2,:]/a_MSR) + y[0,:]))/D_34, # = dy[1,:]/dz
            y[3,:], # = dy[2,:]/dz
            (v*y[3,:] + k_MSR*(y[2,:]**2 + y[0,:]*y[2,:])/((a_MSR*y[0,:]) + y[2,:]))/D_32, # = dy[3,:]/dz
            k_MSR*(y[0,:]*y[2,:] + y[0,:]**2)/(2*v*((y[2,:]/a_MSR) + y[0,:])), # = dy[4,:]/dz
            k_MSR*(y[2,:]**2 + y[0,:]*y[2,:])/(2*v*((a_MSR*y[0,:]) + y[2,:])) # = dy[5,:]/dz
        ])    
    
    def SS_bcs(ya, yb):
        '''The GDE boundary condition residuals, i.e. all the terms involving ya should be
        zero at z=0, and all the terms involving yb should be zero at z=max_z'''
        return np.array([
            ya[0] - SO4_34_0,
            yb[1] - 0,
            ya[2] - SO4_32_0,
            yb[3] - 0,
            ya[4] - py_34_0,
            ya[5] - py_32_0
        ])
    
    # Set up initial array of concentrations to iterate the solution from
    y0s = np.ones((6,len(zs)))
    
    # Use SciPy's boundary value problem solver to solve the steady state ODEs
    SS_sol = solve_bvp(SS_ODEs, SS_bcs, zs, y0s)
    
    # Extract the yi from the solution and return it
    return SS_sol.y



def update_GDE_plot(plot, SO4_0, py_0, d34S_SO4_0, epsilon_MSR, k_MSR, beta, D_32, v, max_z, dz=0.01, plot_time=False):
    
    with plot.batch_update():

        zs = np.arange(0, max_z, dz) # values of z to plot over.

        concs = solve_GDE(zs, SO4_0, py_0, d34S_SO4_0, epsilon_MSR, k_MSR, beta, D_32, v)

        SO4_34s = concs[0,:]
        SO4_32s = concs[2,:]
        py_34s  = concs[4,:]
        py_32s  = concs[5,:]


        # Convert to concentrations and d34S
        SO4s = SO4_34s + SO4_32s
        pys = py_34s + py_32s

        d34S_SO4s = []
        d34S_pys = []

        isotope_tolerance = 1e-12 # mol/dm^3: if numbers below this point, add none to d34S list
        max_SO4_d34S = 100 # don't plot any sulphate isotopes larger than this
        # (n.b. this doesn't affect calculations of other quantities)
        SO4_d34S_valid = True

        for SO4_34, SO4_32, py_34, py_32 in zip(SO4_34s, SO4_32s, py_34s, py_32s):
            if (SO4_32 > isotope_tolerance) and SO4_d34S_valid:
                d34S_SO4 = R_to_delta(SO4_34/SO4_32)
                d34S_SO4s.append(d34S_SO4)
                if d34S_SO4 > max_SO4_d34S: # once threshold is exceeded stop plotting d34S_SO4
                    SO4_d34S_valid = False
            else:
                d34S_SO4s.append(None)

            if py_32 > isotope_tolerance:
                d34S_pys.append(R_to_delta(py_34/py_32))
            else:
                d34S_pys.append(None)

        if plot_time:
            zs = zs/v
            plot.update_layout(yaxis_title='Time after burial, <i>t</i> / ka')
        else:
            plot.update_layout(yaxis_title='Depth, <i>z</i> / m')

        # Get the traces of the graph we want to modify
        traces_list = sel_trace(plot, ['[SO<sub>4</sub>]', '[FeS<sub>2</sub>]', 'δ<sup>34</sup>S<sub>SO4</sub>', 'δ<sup>34</sup>S<sub>py</sub>'])
        # Unpack into their own variables
        SO4_tr, py_tr, d34S_SO4_tr, d34S_py_tr = traces_list

        # Set x coordinates
        SO4_tr.x = SO4s
        py_tr.x = pys
        d34S_SO4_tr.x = d34S_SO4s
        d34S_py_tr.x = d34S_pys

        # Set y coordinates
        for tr in traces_list:
            tr.y = zs

        # final_py, final_d34S = get_py_d34S(SO4_0, py_0, d34S_SO4_0, epsilon_MSR, k_MSR, beta, D_32, v, dz, max_z)
        print('Final [py]: {:.4f}'.format(pys[-1]))
        print('Final d34S: {:.4f}'.format(d34S_pys[-1]))

#### Parameters

Quantity | Name in Code | Value | Unit | Reference
----:----|------:-------|------:|:-----|----:-----
Sulphate concentration in seawater | `SO4_0` | 0.0105 | mol dm<sup>−3</sup> | Horita *et al.* (2002)
Concentration of syn-depositional pyrite in sediment | `py_0` | ?? | mol dm<sup>−3</sup> |
Isotopic composition of seawater sulphate | `d34S_SO4_0` | 15 ~ 17 | ‰ | Kampschulte & Strauss (2004)
Microbial sulphate reduction fractionation amount at surface |`epsilon_MSR` | 0 ~ 66 | ‰ | Fike *et al.* (2016)
Rate constant for sulphate reduction | `k_MSR` | 7 | ka<sup>−1</sup> | Egger *et al.* (2016)
Amount of mass dependency for diffusion coefficients | `beta` | 0 ~ 0.05 | (dimensionless) | Wortmann & Chernyavsky (2011)
Diffusivity of <sup>32</sup>SO<sub>4</sub> | `D_32` | ≈ 0.126 | m<sup>2 </sup>ka<sup>−1</sup> | Jørgensen (1979)
Sedimentation rate | `v` | ≈ 0.08 | m ka<sup>−1</sup> | Gallois (2004)
Max depth to run simulation to | `max_z` | (until δ<sup>34</sup>S<sub>py</sub> stabilises)| m |

In [6]:
GDE_plot = go.FigureWidget(make_subplots(rows=1, cols=2, shared_yaxes=True, horizontal_spacing=0.01))
GDE_plot.add_trace(go.Scatter(name='[SO<sub>4</sub>]', marker_color='RoyalBlue', mode='lines'), row=1, col=1)
GDE_plot.add_trace(go.Scatter(name='[FeS<sub>2</sub>]', marker_color='MediumSeaGreen', mode='lines'), row=1, col=1)
GDE_plot.add_trace(go.Scatter(name='δ<sup>34</sup>S<sub>SO4</sub>', marker_color='Navy', mode='lines'), row=1, col=2)
GDE_plot.add_trace(go.Scatter(name='δ<sup>34</sup>S<sub>py</sub>', marker_color='DarkGreen', mode='lines'), row=1, col=2)

GDE_plot.update_layout(
    yaxis=dict(autorange='reversed', rangemode='tozero', title='Depth, <i>z</i> / m'),
    hovermode='y',
    margin=dict(t=60),
    width=1000, height=600
    )

GDE_plot.update_xaxes(title_text='Concentration / mol dm<sup>−3</sup>', row=1, col=1)
GDE_plot.update_xaxes(title_text='δ<sup>34</sup>S (‰)', row=1, col=2)

# create widgets
w_d = create_sliders(w_df, prefix='w')

w_plot_time = wdg.Checkbox(value=False, description='Plot time on y axis')

# set up widget layout
GDE_ui = wdg.VBox([wdg.HBox([w_d['SO4_0'], w_d['d34S_SO4_0']]),
                   wdg.HBox([w_d['py_0'], w_d['v']]),
                   wdg.HBox([w_d['epsilon_MSR'], w_d['k_MSR']]),
                   wdg.HBox([w_d['D_32'], w_d['beta']]),
                   wdg.HBox([w_d['max_z'], w_d['dz']]),
                   w_plot_time
                  ])

GDE_args = {**w_d, 
            'plot': wdg.fixed(GDE_plot),
            'plot_time': w_plot_time}

GDE_output = wdg.interactive_output(update_GDE_plot, GDE_args)

display(GDE_ui, GDE_output)

GDE_plot

Output()

FigureWidget({
    'data': [{'marker': {'color': 'RoyalBlue'},
              'mode': 'lines',
              'n…

In [7]:
# GDE_plot.update_xaxes(row=1, col=1, range=[0,0.3])

In [8]:
#GDE_plot.write_image('./images/GDE_example.svg')

In [9]:
def get_py_d34S(SO4_0, py_0, d34S_SO4_0, epsilon_MSR, k_MSR, beta, D_32, v, dz, max_z):
    '''Returns the final pyrite concentration and its d34S given the appropriate initial conditions'''
    
    zs = np.arange(0, max_z, dz) # values of z to plot over.
    
    concs = solve_GDE(zs, SO4_0, py_0, d34S_SO4_0, epsilon_MSR, k_MSR, beta, D_32, v)
    
    # Final amounts of py32 and py34
    py_34  = concs[4,-1]
    py_32  = concs[5,-1]
    
    py = py_34 + py_32
    d34S = R_to_delta(py_34/py_32)
    
    return py, d34S


def update_var_plot(plot, varying, vstart, vstop, vstep, SO4_0, py_0, d34S_SO4_0, epsilon_MSR, k_MSR, beta, D_32, v, dz, max_z, plot_SDP, plot_noSDP):
    
    with plot.batch_update():
    
        # Create dictionary of parameters
        params = {}
        for w in w_df.iterrows():
            param = w[1].var_name
            params[param] = locals()[param] # locals() get the actual variables put into the function

    #     print(params)

        pys = []
        d34Ss = []
        texts = []

        vary_range = np.arange(vstart, vstop, vstep)

        for var in vary_range:

            # Update the appropriate parameter to its value in the iterations
            params[varying] = var

            py, d34S = get_py_d34S(**params)

            pys.append(py)
            d34Ss.append(d34S)
            texts.append('{}: {:.5f}'.format(varying, var))

    #         print('{}: {:.5f}'.format(varying, vars()[varying]))
    #         print('[py]: {:.4f}'.format(py))
    #         print('d34S: {:.4f}\n'.format(d34S))

        model_tr = sel_trace(plot, 'Model')

        # plot.update_traces(x=pys, y=d34Ss, text=texts, marker_color=vary_range, selector=dict(name='Model'))
        model_tr.x = pys
        model_tr.y = d34Ss
        model_tr.text = texts
        model_tr.marker.color = vary_range

        SDP_tr = sel_trace(plot, 'Syn-Depositional Pyrite')
        if plot_SDP:
            SDP_tr.x = [py_0]
            SDP_tr.y = [d34S_SO4_0 - epsilon_MSR]
            SDP_tr.text = ['{:.2f} wt% Fe<sub>py, 0</sub>'.format(M_to_wt(py_0))]
            SDP_tr.visible = True

        else:
            SDP_tr.visible = False


        noSDP_tr = sel_trace(plot, 'Model (no py<sub>0</sub>)')
        if plot_noSDP:
            # recalculate model with no syn-depositional pyrite
            pys_na = []
            d34Ss_na = []
            texts_na = []

            params_na = params
            params_na['py_0'] = 0 # set to no syn-depositional pyrite

            for var in vary_range:

                # Update the appropriate parameter to its value in the iterations
                params_na[varying] = var

                py, d34S = get_py_d34S(**params_na)

                pys_na.append(py)
                d34Ss_na.append(d34S)
                texts_na.append('{}: {:.5f}'.format(varying, var))

    #         plot.update_traces(x=pys_na, y=d34Ss_na, text=texts_na, marker_color=vary_range,
    #                            visible=True, selector=dict(name='Model (no py<sub>0</sub>)'))
            noSDP_tr.x = pys_na
            noSDP_tr.y = d34Ss_na
            noSDP_tr.text = texts_na
            noSDP_tr.marker.color = vary_range
            noSDP_tr.visible = True

        else:
            noSDP_tr.visible = False

        # change colour bar title
        plot.layout.coloraxis.colorbar.title = list(w_df[w_df.var_name==varying].display_name)[0]

In [10]:
# Set up plot
var_plot=go.FigureWidget()

var_plot.update_layout(
    yaxis=dict(title='δ<sup>34</sup>S<sub>py</sub> (‰)'),
    xaxis=dict(title='[FeS<sub>2</sub>] / mol dm<sup>−3</sup>', rangemode='tozero'),
    legend=dict(yanchor="top", y=1, xanchor="left", x=1.2),
    margin=dict(l=100),
    width=1000, height=600,
    coloraxis=dict(colorscale='viridis_r', colorbar_ticks='outside')
    )

# Add the real data
for category in categories:
    cat_df = log_df[log_df.category == category]
    var_plot.add_trace(go.Scatter(x = cat_df.Fe_py.apply(wt_to_M), y=cat_df.d34S,
                                  text = cat_df.name,
                                  mode='markers',
                                  marker_color='Blue',
                                  marker_opacity=0.3,
                                  marker_symbol=cat_symbols[category],
                                  name=category))

var_plot.add_trace(go.Scatter(name='Model', mode='lines+markers', marker_coloraxis='coloraxis',
                              marker_line_color='DarkGreen', marker_line_width=1, line_color='DarkGreen'))
var_plot.add_trace(go.Scatter(name='Model (no py<sub>0</sub>)', mode='lines+markers', marker_coloraxis='coloraxis',
                              marker_line_color='Orange', marker_line_width=1, line_color='Orange'))
var_plot.add_trace(go.Scatter(name='Syn-Depositional Pyrite', mode='markers',
                              marker_color='Purple', marker_symbol='square'))

# create widgets
field_layout = wdg.Layout(width='200px')

v_d = create_sliders(w_df, prefix='v')
v_varying=wdg.Dropdown(options=w_df.var_name, description='Vary', layout=field_layout, value='v')
v_vstart=wdg.FloatText(value=0.005, step=0.01, description='from', layout=field_layout)
v_vstop=wdg.FloatText(value=0.2, step=0.1, description='to', layout=field_layout)
v_vstep=wdg.FloatText(value=0.005, step=0.01, description='step', layout=field_layout)
v_plot_SDP = wdg.Checkbox(value=True, description='Plot syn-depositional pyrite')
v_plot_noSDP = wdg.Checkbox(value=False, description='Plot curve w/o syn-depositional pyrite')

# set up widget layout
var_ui = wdg.VBox([wdg.HBox([v_d['SO4_0'], v_d['d34S_SO4_0']]),
                   wdg.HBox([v_d['py_0'], v_d['v']]),
                   wdg.HBox([v_d['epsilon_MSR'], v_d['k_MSR']]),
                   wdg.HBox([v_d['D_32'], v_d['beta']]),
                   wdg.HBox([v_d['max_z'], v_d['dz']]),
                   wdg.HBox([v_varying, v_vstart, v_vstop, v_vstep]),
                   wdg.HBox([v_plot_SDP, v_plot_noSDP])
                  ])

var_args = {**v_d, 
            'plot': wdg.fixed(var_plot),
            'varying':v_varying,
            'vstart':v_vstart,
            'vstop':v_vstop,
            'vstep':v_vstep,
            'plot_SDP': v_plot_SDP,
            'plot_noSDP': v_plot_noSDP
           }

var_output = wdg.interactive_output(update_var_plot, var_args)

display(var_ui, var_output)

var_plot

Output()

FigureWidget({
    'data': [{'marker': {'color': 'Blue', 'opacity': 0.3, 'symbol': 'circle'},
              'm…

In [11]:
# var_plot.write_image('./images/varying_SO4.svg')

### Modelling Iron Speciation with the same parameters

In [12]:
def update_Fe_spec(plot, SO4_0, py_0, d34S_SO4_0, epsilon_MSR, k_MSR, beta, D_32, v, dz, max_z, Fe_tot, plot_residuals):
    
    with plot.batch_update():

        # Calculate how much pyrite is produced
        py, d34S = get_py_d34S(SO4_0, py_0, d34S_SO4_0, epsilon_MSR, k_MSR, beta, D_32, v, dz, max_z)
        Fe_py = M_to_wt(py)
        Fe_py_0 = M_to_wt(py_0)

        # update model line
        HR_tot_0 = np.linspace(0.01,1,50) # Increase last number for higher resolution

        HR_0 = HR_tot_0 * Fe_tot
        py_HR_t = py/HR_0
        
        model_tr = sel_trace(plot, 'model (const. total Fe)')
        model_tr.x = HR_tot_0
        model_tr.y = py_HR_t


        # update data points
        for category in categories:
            cat_df = log_df[log_df.category == category]

            py_HR_points = list(Fe_py/cat_df['Fe_HR'])

            py_0_HR = list(Fe_py_0/cat_df['Fe_HR'])

            for i in range(len(py_HR_points)):
                if py_HR_points[i] > 1:
                    py_HR_points[i] = 1
                    
            modelled_tr = sel_trace(plot, category+' (modelled)')
            modelled_tr.x = cat_df['HR_to_tot']
            modelled_tr.y = py_HR_points

            plot.update_traces(selector=dict(name=category+' (pre-diagenesis)'),
                               x=cat_df['HR_to_tot'], y=py_0_HR)
            
            pre_dia_tr = sel_trace(plot, category+' (pre-diagenesis)')
            pre_dia_tr.x = cat_df['HR_to_tot']
            pre_dia_tr.y = py_0_HR
            
            residuals = sel_trace(plot, category+' (residuals)')
            if plot_residuals:
                residuals.x = cat_df['HR_to_tot']
                residuals.y = cat_df['py_to_HR'] - py_HR_points
                residuals.visible = True
            else:
                residuals.visible = False

#         if plot_residuals:
#             plot.update_layout(yaxis_range=[-0.5,1])
#         else:
#             plot.update_layout(yaxis_range=[0,1])
    
    # plot.show(config=graph_config)

In [13]:
# Modelling pyritisation of real data
psn_model=go.FigureWidget()

# for HR_to_tot in list(log_df.HR_to_tot):
#     psn_model.add_vline(HR_to_tot, line_width=0.5, line_color='Lavender')

for category in categories: 
    
    cat_df = log_df[log_df.category == category]

    graph_texts = []
    for d in cat_df.iterrows():
        graph_texts.append(d[1]['name'])
    
    # Current locations of samples in Fe speciation space
    
    psn_model.add_trace(go.Scatter(mode='markers',
                                   marker_symbol=cat_symbols[category],
                                   marker_color='Purple',
                                   marker_opacity=0.3,
                                   name=category+' (pre-diagenesis)',
                                   text=graph_texts,
                                   legendgroup=category))
    
    psn_model.add_trace(go.Scatter(mode='markers',
                                   marker_symbol=cat_symbols[category],
                                   marker_color='Green',
                                   name=category+' (modelled)',
                                   text=graph_texts,
                                   legendgroup=category))

    psn_model.add_trace(go.Scatter(x=cat_df['HR_to_tot'], y=cat_df['py_to_HR'],
                                   mode='markers',
                                   marker_symbol=cat_symbols[category],
                                   marker_color='Blue',
                                   marker_opacity=0.3,
                                   name=category+' (post-diagenesis)',
                                   text=graph_texts,
                                   legendgroup=category))    
    
    psn_model.add_trace(go.Scatter(mode='markers',
                                   marker_symbol=cat_symbols[category],
                                   marker_color='FireBrick',
                                   name=category+' (residuals)',
                                   text=graph_texts,
                                   legendgroup=category))
    
    
psn_model.add_trace(go.Scatter(name='model (const. total Fe)',
                               line_color='LightGreen'))

psn_model.update_layout(xaxis_range=[0,1], yaxis_range=[0,1],
                        width=900, height=600)
psn_model.add_hline(0.8, line_width=1, line_dash='dash')
psn_model.add_vline(0.38, line_width=1, line_dash='dash')
psn_model.update_layout(xaxis_title='[Fe<sub>HR</sub>]/[Fe<sub>tot</sub>]',
                        yaxis_title='[Fe<sub>py</sub>]/[Fe<sub>HR</sub>]',
                        legend=dict(#itemsizing='constant',
                                    font_size=11))


# Create widgets
s_d = create_sliders(w_df, prefix='s')
s_Fe_tot=wdg.FloatSlider(value=4, min=0, max=15, description='[Fe<sub>tot</sub>]', layout=wdg.Layout(width='100%')) # Total Fe for simple model curve
s_plot_residuals=wdg.Checkbox(value=False, description='Plot Residuals')

spec_ui = wdg.VBox([wdg.HBox([s_d['SO4_0'], s_d['d34S_SO4_0']]),
                   wdg.HBox([s_d['py_0'], s_d['v']]),
                   wdg.HBox([s_d['epsilon_MSR'], s_d['k_MSR']]),
                   wdg.HBox([s_d['D_32'], s_d['beta']]),
                   wdg.HBox([s_d['max_z'], s_d['dz']]),
                   wdg.HBox([s_Fe_tot, s_plot_residuals])])

spec_args={**s_d, 'Fe_tot': s_Fe_tot, 'plot_residuals': s_plot_residuals, 'plot': wdg.fixed(psn_model)}

spec_out = wdg.interactive_output(update_Fe_spec, spec_args)

display(spec_ui, spec_out)

psn_model

Output()

FigureWidget({
    'data': [{'legendgroup': 'Mudstone',
              'marker': {'color': 'Purple', 'opacity':…

## Monte-Carlo Simulation

In [14]:
def create_range_sliders(w_df, prefix='wr'):
    w_dict = {}

    for w in w_df.iterrows():

        # Make a new variable for each widget
        vars()[prefix+'_'+w[1].var_name] = wdg.FloatRangeSlider(value = [w[1].dr_min, w[1].dr_max],
                                                                min = w[1].start,
                                                                max = w[1].stop,
                                                                step = w[1].step,
                                                                description = w[1].display_name,
                                                                layout=wdg.Layout(width='100%')
                                                               )

        # Add to a dictionary of original variables and widgets
        w_dict[w[1].var_name] = vars()[prefix+'_'+w[1].var_name]
    
    return w_dict

In [15]:
# Dataframe of only parameters we want to vary in Monte-Carlo simulation
monte_df = w_df[~w_df.var_name.isin(['max_z', 'dz'])]
monte_df

,var_name,value,start,stop,step,display_name,dr_min,dr_max
0,SO4_0,0.0105,0.001,0.30,0.0010,[SO<sub>4</sub>]<sub>0</sub>,0.008,0.013
1,py_0,0.1600,0.000,1.00,0.0025,[FeS<sub>2</sub>]<sub>0</sub>,0.100,0.250
2,d34S_SO4_0,23.0000,0.000,30.00,0.1000,"δ<sup>34</sup>S<sub>SO4, 0</sub>",14.000,18.000
3,epsilon_MSR,54.0000,0.000,100.00,1.0000,ε<sub>MSR</sub>,40.000,55.000
4,k_MSR,10.0000,0.000,40.00,0.1000,k<sub>MSR</sub>,5.000,15.000
5,v,0.1200,0.001,0.50,0.0010,v<sub>sed</sub>,0.005,0.200
6,D_32,0.2000,0.001,0.30,0.0010,D<sub>32</sub>,0.100,0.300
7,beta,0.0250,0.000,0.05,0.0010,β,0.000,0.050


In [16]:
def update_monte_plot(plot, SO4_0, py_0, d34S_SO4_0, epsilon_MSR, k_MSR, beta, D_32, v, dz, max_z, n_trials): # , colour_param):
    
    with plot.batch_update():
    
        params_list = []

#         pys = []
#         d34Ss = []
#         texts = []
#         colours = []
        
        # Loop for each trial in Monte Carlo
        for trial in range(n_trials):
            
            # Create dictionary for this run's parameters
            params = {}
            
            # Randomise the parameters
            for row in monte_df.iterrows():
                param_name = row[1].var_name
                param_range = locals()[param_name] # locals() returns dictionary of the actual variables put into the function
                
                param = random.uniform(*param_range) # Picks a random float within the parameter range (inclusive)
                
                # Update the appropriate parameter in the dictionary
                params[param_name] = param

            # After randomising all parameters, put them into the function (and add those we didn't randomise)
            py, d34S = get_py_d34S(**params, dz=dz, max_z=max_z)
            
#             params['Trial_No'] = trial
            params['py'] = py
            params['d34S'] = d34S

#             pys.append(py)
#             d34Ss.append(d34S)
#             texts.append(f'Trial {trial}')
#             colours.append(params[colour_param])
            
            params_list.append(params)

        # After iteration, convert the parameter list into a dataframe, stroed in the global variable params_df
        global params_df
        params_df = pd.DataFrame(params_list)
        
        
        model_tr = sel_trace(plot, 'Trials')

        model_tr.x = params_df.py
        model_tr.y = params_df.d34S
        
        # Get the parameter used for colour without forcing it to update this function
        model_tr.marker.color = params_df[r_colour.value]
        
        model_tr.text = params_df.index


#         SDP_tr = sel_trace(plot, 'Syn-Depositional Pyrite')
#         if plot_SDP:
#             SDP_tr.x = [py_0]
#             SDP_tr.y = [d34S_SO4_0 - epsilon_MSR]
#             SDP_tr.text = ['{:.2f} wt% Fe<sub>py, 0</sub>'.format(M_to_wt(py_0))]
#             SDP_tr.visible = True

#         else:
#             SDP_tr.visible = False


#         noSDP_tr = sel_trace(plot, 'Model (no py<sub>0</sub>)')
#         if plot_noSDP:
#             # recalculate model with no syn-depositional pyrite
#             pys_na = []
#             d34Ss_na = []
#             texts_na = []

#             params_na = params
#             params_na['py_0'] = 0 # set to no syn-depositional pyrite

#             for var in vary_range:

#                 # Update the appropriate parameter to its value in the iterations
#                 params_na[varying] = var

#                 py, d34S = get_py_d34S(**params_na)

#                 pys_na.append(py)
#                 d34Ss_na.append(d34S)
#                 texts_na.append('{}: {:.5f}'.format(varying, var))

#     #         plot.update_traces(x=pys_na, y=d34Ss_na, text=texts_na, marker_color=vary_range,
#     #                            visible=True, selector=dict(name='Model (no py<sub>0</sub>)'))
#             noSDP_tr.x = pys_na
#             noSDP_tr.y = d34Ss_na
#             noSDP_tr.text = texts_na
#             noSDP_tr.marker.color = vary_range
#             noSDP_tr.visible = True

#         else:
#             noSDP_tr.visible = False

        # change colour bar title
#         plot.layout.coloraxis.colorbar.title = list(monte_df[monte_df.var_name==colour_param].display_name)[0]

In [17]:
def tweak_plot(marker_size, marker_opacity, colour_param, trace, plot):
    trace.marker.size = marker_size
    trace.marker.opacity = marker_opacity
    trace.marker.color = params_df[colour_param]
    plot.layout.coloraxis.colorbar.title = list(monte_df[monte_df.var_name==colour_param].display_name)[0]

In [18]:
r_details = wdg.Output()

def monte_hover(trace, points, state):
    if points.point_inds: # Makes sure we're hovering over the right trace. The wrong trace will give point_inds = [].
        ind = points.point_inds[0]
        
        details = params_df.iloc[ind].to_frame() # creates summary dataframe
        
        with r_details:
            display(details) # displays dataframe in output widget
            
        r_details.clear_output(wait=True) # Clears the output when a new point is selected.
    else:
        pass

In [21]:
# Set up plot
monte_plot=go.FigureWidget()

monte_plot.update_layout(yaxis=dict(title='δ<sup>34</sup>S<sub>py</sub> (‰)'),
                         xaxis=dict(title='[FeS<sub>2</sub>] / mol dm<sup>−3</sup>', rangemode='tozero'),
                         legend=dict(yanchor="top", y=1, xanchor="left", x=1.2),
                         margin=dict(l=100),
                         width=1000, height=600,
                         coloraxis=dict(colorscale='viridis_r', colorbar_ticks='outside')
                         )

# Add the real data
for category in categories:
    cat_df = log_df[log_df.category == category]
    monte_plot.add_scatter(x = cat_df.Fe_py.apply(wt_to_M), y=cat_df.d34S,
                           text = cat_df.name,
                           mode='markers',
                           marker_color='Blue',
                           marker_opacity=0.3,
                           marker_symbol=cat_symbols[category],
                           name=category)

monte_plot.add_trace(go.Scatter(name='Trials', mode='markers', marker_coloraxis='coloraxis',
                                marker_line={'color': 'Black', 'width': 1}))
# monte_plot.add_trace(go.Scatter(name='Model (no py<sub>0</sub>)', mode='lines+markers', marker_coloraxis='coloraxis',
#                                 marker_line_color='Orange', marker_line_width=1, line_color='Orange'))
# monte_plot.add_trace(go.Scatter(name='Syn-Depositional Pyrite', mode='markers',
#                                 marker_color='Purple', marker_symbol='square'))

# create widgets
field_layout = wdg.Layout(width='200px')

r_d = create_range_sliders(monte_df, prefix='r')
r_d2 = create_sliders(w_df[w_df.var_name.isin(['max_z', 'dz'])], prefix='r')

r_n_trials = wdg.IntSlider(value = 400, min = 1, max = 1000, description = '№ of Trials', layout=wdg.Layout(width='100%'))

r_colour=wdg.Dropdown(options=monte_df.var_name, description='Colour', layout=field_layout, value='v')
r_marker_size = wdg.FloatSlider(value=8, min=0, max=20, step=0.1, description='Marker Size', layout=wdg.Layout(width='100%'))
r_opacity = wdg.FloatSlider(value=0.7, min=0, max=1, step=0.01, description='Opacity', layout=wdg.Layout(width='100%'))

# v_plot_SDP = wdg.Checkbox(value=True, description='Plot syn-depositional pyrite')
# v_plot_noSDP = wdg.Checkbox(value=False, description='Plot curve w/o syn-depositional pyrite')

# set up widget layout
monte_ui = wdg.VBox([wdg.HBox([r_d['SO4_0'], r_d['d34S_SO4_0']]),
                     wdg.HBox([r_d['py_0'], r_d['v']]),
                     wdg.HBox([r_d['epsilon_MSR'], r_d['k_MSR']]),
                     wdg.HBox([r_d['D_32'], r_d['beta']]),
                     wdg.HBox([r_d2['max_z'], r_d2['dz']]),
                     wdg.HBox([r_n_trials]),
                     wdg.HBox([r_colour, r_marker_size, r_opacity])
#                      wdg.HBox([v_plot_SDP, v_plot_noSDP])
                    ])

monte_args = {**r_d, **r_d2,
              'plot': wdg.fixed(monte_plot),
              'n_trials': r_n_trials,
#              'colour_param': r_colour,
#               'plot_SDP': v_plot_SDP,
#               'plot_noSDP': v_plot_noSDP
             }

monte_output = wdg.interactive_output(update_monte_plot, monte_args)

tweak_args = {'trace': wdg.fixed(sel_trace(monte_plot, 'Trials')),
              'plot': wdg.fixed(monte_plot),
              'marker_size': r_marker_size,
              'marker_opacity': r_opacity,
              'colour_param': r_colour
             }
tweak_output = wdg.interactive_output(tweak_plot, tweak_args)

sel_trace(monte_plot, 'Trials').on_hover(monte_hover)

monte_dashboard = wdg.VBox([monte_ui,
                            wdg.HBox([monte_plot, r_details])
                           ])

display(monte_dashboard, monte_output, tweak_output)

Output()

Output()

In [20]:
monte_plot.write_image('images/monte_plot_test.svg')